In [1]:
import tensorflow as tf
import numpy as np
import os, sys, string, random

from PIL import Image

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

%matplotlib inline

In [2]:
# Get a list of the sub-directories in the INPUT_DIRECTORY
INPUT_BG_DIRECTORY = os.path.join("data_bg", "cleaned")

bg_directories  = [ name for name in os.listdir(INPUT_BG_DIRECTORY) if os.path.isdir(os.path.join(INPUT_BG_DIRECTORY, name)) ]

# We scan all the files in the sub-directories with the extensions given above
all_bg_files = tf.concat(
    [tf.train.match_filenames_once(INPUT_BG_DIRECTORY + "/" + x + "/*.png") for x in bg_directories],
    0
)

all_bg_files_length = tf.shape(all_bg_files)[0]

In [24]:
def color_to_alpha(image, luminance_threshold=70):
    
    data = np.array(image)
    
    rgb = data[:,:,:3]
    
    mask = list()

    for row in rgb:
        row_arr = list()

        for pix in row:
            lum = ((0.2126*pix[0]) + (0.7152*pix[1]) + (0.0722*pix[2]))/255*100
            row_arr.append(lum >= luminance_threshold)

        mask.append(row_arr)

    mask = np.array(mask)

    transparent_color = [0, 0, 0, 0]
    
    # change all pixels that match color to transparent_color*
    data[mask] = transparent_color
    
    return data

In [25]:
def overlay_image2background(image, bg_filename):
    bg = Image.open(bg_filename)
    bg = bg.convert("RGBA")
    
    composed_img = Image.alpha_composite(bg, Image.fromarray(image))
    
    return composed_img.convert("RGB")

In [26]:
filename_queue = tf.train.string_input_producer(tf.train.match_filenames_once("./random_dir/original/*.png"))

In [27]:
image_reader = tf.WholeFileReader()

In [28]:
_, image_file = image_reader.read(filename_queue)

In [29]:
rand_idx = tf.random_uniform(
    [1], 
    minval = 0, 
    maxval = all_bg_files_length - 1,
    dtype  = tf.int32
)[0]

bg_filename = all_bg_files[rand_idx]

In [30]:
image               = tf.image.decode_png(image_file, channels=4)

image_processed     = tf.py_func(color_to_alpha, [image], tf.uint8)
image_composed      = tf.py_func(overlay_image2background, [image_processed, bg_filename], tf.uint8)

image_processed_enc = tf.image.encode_png(image_processed)
image_composed_enc  = tf.image.encode_png(image_composed)

In [31]:
init_op_global = tf.global_variables_initializer()
init_op_local = tf.local_variables_initializer()

In [32]:
def id_generator(size=20, chars=string.ascii_uppercase + string.ascii_lowercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

In [33]:
out_dir = "random_dir/output"

with tf.Session() as sess:
    # Required to get the filename matching to run.
    sess.run([init_op_global, init_op_local])

    # Coordinate the loading of image files.
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    # Get an image tensor and print its value.
    _image_processed = sess.run(image_composed_enc)
    
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
        
    filename = out_dir + "/" + id_generator() + ".png"

    with open(filename, "wb+") as f:
        f.write(_image_processed)
        f.close()

    # Finish off the filename queue coordinator.
    coord.request_stop()
    coord.join(threads)